In [1]:
import glob,os
from nltk.tokenize import word_tokenize
import operator

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier

In [2]:
path = './data'
data = []
for infile in glob.glob( os.path.join(path, '*.*') ):
       with open(infile) as file:
            string = file.readlines()
            for each_line in string:
                if len(each_line.split())>5: # to remove time and name from the doc
                    data.extend(each_line.replace('\n','').split(":")[2:])

In [3]:
# Importing files containing Ques, Ans and Others data
ques_cluster = open('./Clusters/ques.txt','r').readlines()
ans_cluster = open('./Clusters/ans.txt','r').readlines()
others_cluster = open('./Clusters/others.txt','r').readlines()

In [4]:
# List of all Concatenated Questions, Concatenated Answers, Concatenated Others
Questions_String = [" ".join([i for i in ques_cluster])]
Answers_String = [" ".join([i for i in ans_cluster])]
Others_String = [" ".join([i for i in others_cluster])]

# List of ques, ans and others
ques_list = [i for i in ques_cluster]
ans_list = [i for i in ans_cluster]
others_list = [i for i in others_cluster]

# List containing All concatenated Questions, list of Answers and List of Others
AllQuestion_ans_others = Questions_String + ans_list + others_list #[Q,A,O,A,O,A,A,O]
AllAnswer_ques_others = Answers_String + ques_list + others_list
AllOthers_ans_ques = Others_String + ans_list + ques_list

## Questions Feature

In [5]:
stop_words = ['ask','create','task','checked','fit','let','tell','number', 'specialist' , 'support' , 'technical' , 'today' , 'welcome','select','test','using','able','check','fix','found','gives','something','tried','wrong','even','happens','normal','solve','step','though','when','back','wait','start','steps','face','issue','suggest','please','use','wall','available','contact','face','feel','point','free','support','time','us','problem','goes','anything','cant','goes','lets','put','wont','akash','13th','able','black','went','last','made','when','wrong','www','type','please','enter','i','me', 'my', 'myself','we','our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their','theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being','have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing','a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for','with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'all','any', 'both', 'each', 'few','more', 'most', 'other','some','such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will','just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn','weren','won','wouldn']

In [6]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(AllQuestion_ans_others).todense() #fit the vectorizer to synopses

In [7]:
ques_features = list(zip(tfidf_vectorizer.get_feature_names(), tfidf_matrix.tolist()[0]))
ques_features = sorted(ques_features, key=lambda x: x[1], reverse= True)
ques_features = [i for i in ques_features if i[1]>0]

## Answers Features

In [8]:
#define vectorizer parameters
tfidf_vectorizer_ans = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix_ans = tfidf_vectorizer_ans.fit_transform(AllAnswer_ques_others).todense() #fit the vectorizer to synopses

In [9]:
ans_features = list(zip(tfidf_vectorizer_ans.get_feature_names(), tfidf_matrix_ans.tolist()[0]))
ans_features = sorted(ans_features, key=lambda x: x[1], reverse= True)
ans_features = [i for i in ans_features if i[1]>0]

## Others Features

In [10]:
#define vectorizer parameters
tfidf_vectorizer_others = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix_others = tfidf_vectorizer_others.fit_transform(AllOthers_ans_ques).todense() #fit the vectorizer to synopses

In [11]:
others_features = list(zip(tfidf_vectorizer_others.get_feature_names(), tfidf_matrix_others.tolist()[0]))
others_features = sorted(others_features, key=lambda x: x[1], reverse= True)
others_features = [i for i in others_features if i[1]>0]

## All Features in Corpus 

In [12]:
all_features = [i[0] for i in ques_features] + [i[0] for i in ans_features] + [i[0] for i in others_features]
all_features = list(set(all_features))
all_features = [x for x in all_features if not x.strip().isdigit() ]

## Frequency Matrix

In [13]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(all_features)
print("Vocabulary:", count_vectorizer.vocabulary_)

Vocabulary: {'pushed': 241, 'software': 303, 'ended': 103, 'assistance': 30, 'husband': 149, 'okay': 203, 'page': 213, 'honestly': 142, 'background': 35, 'timezone': 326, 'disconnect': 92, 'particular': 216, 'possible': 224, 'little': 167, 'share': 289, 'app04': 19, 'app': 17, 'ups': 341, 'minutes': 184, 'regarding': 256, 'elaborate': 101, 'cancel': 49, 'place': 222, 'almost': 12, 'thing': 323, 'booting': 43, 'device': 88, 'msconfig': 189, 'like': 166, 'voice': 348, 'sent': 281, 'provide': 240, 'reboots': 250, 'sharing': 290, 'hapends': 132, 'reason': 248, 'long': 174, 'soon': 304, 'canelled': 50, 'call': 48, 'shoot': 291, 'another': 15, 'remote': 258, 'hardware': 134, 'connect': 70, 'message': 181, 'asap': 27, 'extemely': 112, 'booted': 42, 'took': 327, 'privacy': 230, 'application': 23, 'interfere': 156, 'ushasi': 344, 'practices': 227, 'either': 100, 'compatibility': 64, 'takes': 320, 'settings': 287, 'need': 193, 'koushik': 161, 'issues': 157, 'allowing': 11, 'updated': 339, 'infor

In [14]:
def train_test(ques_list,ans_list,others_list,test_size = 0.25):
    train_ques,test_ques = train_test_split(ques_list,test_size=test_size)
    ques_train_class_list, ques_test_class_list = train_test_split(["Ques"]*len(ques_list),test_size=test_size)
    train_ans,test_ans = train_test_split(ans_list,test_size=test_size)
    ans_train_class_list, ans_test_class_list = train_test_split(["Ans"]*len(ans_list),test_size=test_size)
    train_other,test_other = train_test_split(others_list,test_size=test_size)
    other_train_class_list, other_test_class_list = train_test_split(["Others"]*len(others_list),test_size=test_size)
    
    train_data = train_ques + train_ans + train_other
    test_data =  test_ques + test_ans + test_other
    
    class_list_train = ques_train_class_list + ans_train_class_list + other_train_class_list 
    class_list_test = ques_test_class_list + ans_test_class_list + other_test_class_list
    return train_data, test_data, class_list_train, class_list_test

In [15]:
train_data, test_data, class_list_train, class_list_test = train_test(ques_list, ans_list, others_list, 0.3)

## Grid Search-SKLEARN to select HyperParameter

In [16]:
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators' : [150] ,'max_features' : [75] ,'max_depth' : [20],'max_leaf_nodes' : [12]}]

In [17]:
rf_classifier = RandomForestClassifier(n_estimators=100, max_features='auto',max_depth=10, max_leaf_nodes=3)

In [18]:
CV_rfc = GridSearchCV(estimator=rf_classifier, param_grid = tuned_parameters, cv= 10)
CV_rfc.fit(count_vectorizer.transform(train_data), class_list_train)
print(CV_rfc.best_params_)

{'max_depth': 20, 'max_features': 75, 'max_leaf_nodes': 12, 'n_estimators': 150}


In [19]:
y_true, y_pred = class_list_test, CV_rfc.predict(count_vectorizer.transform(test_data))
print(classification_report(y_true, y_pred))
print()

             precision    recall  f1-score   support

        Ans       1.00      0.11      0.20         9
     Others       0.93      0.37      0.53        35
       Ques       0.58      0.98      0.73        43

avg / total       0.77      0.64      0.60        87




In [20]:
# for i, sentence in enumerate(test_data):
#     if(CV_rfc.predict(count_vectorizer.transform([sentence])) == class_list_test[i]):
#         print(sentence," ::" ,CV_rfc.predict(count_vectorizer.transform([sentence])),"::", class_list_test[i])

In [21]:
# for i, sentence in enumerate(test_data):
#     if(CV_rfc.predict(count_vectorizer.transform([sentence])) != class_list_test[i]):
#         print(sentence," ::" ,CV_rfc.predict(count_vectorizer.transform([sentence])),"::", class_list_test[i])

In [22]:
ordered_features = [key for key, value in sorted(count_vectorizer.vocabulary_.items(), key=operator.itemgetter(1))]

In [23]:
def get_class_features(list_of_sentences, count_vectorizer = count_vectorizer, ordered_features = ordered_features, rf = CV_rfc ):
    
    class_of = count_vectorizer.transform(list_of_sentences)
    class_is = rf.predict(class_of)
    
    features = []
    for each_class in class_of.todense().tolist():
        test = list(zip(ordered_features, each_class))
        features.append([i for i in test if i[1]>0])
    
    class_feature = []
    for index, each_sentence in enumerate(list_of_sentences):
        class_feature.append({"query":each_sentence, "class": class_is[index], "feature": [i[0] for i in features[index]]})
    return class_feature

In [24]:
input_document = ["Do i ask check the create this task with administrative privileges?", "My laptop is not working.","How are you?","While troubleshoot I checked the device drivers and software and let me tell you, your HP laptop is just fit and dandy."]

In [25]:
get_class_features(input_document)

[{'class': 'Ques',
  'feature': ['administrative', 'privileges'],
  'query': 'Do i ask check the create this task with administrative privileges?'},
 {'class': 'Ques',
  'feature': ['laptop', 'working'],
  'query': 'My laptop is not working.'},
 {'class': 'Others', 'feature': ['how'], 'query': 'How are you?'},
 {'class': 'Ques',
  'feature': ['dandy',
   'device',
   'drivers',
   'hp',
   'laptop',
   'software',
   'troubleshoot'],
  'query': 'While troubleshoot I checked the device drivers and software and let me tell you, your HP laptop is just fit and dandy.'}]